In [3]:
from helpers import *
import pandas as pd 
import numpy as np 
import re
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_app3")

import time

In [4]:
#Realizar Web Scraping 
"""
Region: Recomendable escribir nombre de region igual al configurado  por defecto en la pagina de portal inmobiliario
        fijarse en url de busqueda, por ejemplo: al escribir santiago en buscador principal, la pagina redirige a los resultados con la         categoria "santiago-metropolitana"
pagina principal: https://www.portalinmobiliario.com/
ejemplo nombre de region url: https://www.portalinmobiliario.com/arriendo/departamento/santiago-metropolitana

Pages: Cantidad de paginas de resultados para la busqueda de ofertas,  no se puede superar el numero de paginas de resultados que existen en portalinmobiliario según la region especificada.

Scopes: pueden ser arriendo o venta.

type: puede ser departamento, casa, sitio, parcela, oficina, comercial, industrial, agricola, terreno-en-construccion, bodega, estacionamiento,  loteo, otros 
(considerar que webscraping esta optimizado para departamentos, por lo que puede que al utilizar otro tipo de inmueble algunas columnas como baños, habitaciones,etc podrian no coincidir, debido a que la información para cada tipo de inmueble es distinta)
"""

df = webscraping_deptos(region="santiago-metropolitana",pages=1,type='departamento',scope='arriendo')
print(df)

Web Scraping Portal Inmobiliario
Buscando departamento para arriendo....
https://www.portalinmobiliario.com/arriendo/departamento/santiago-metropolitana/_Desde_1


NameError: name 'driver' is not defined

In [3]:
from datetime import datetime

now = datetime.now()
formatted_time = now.strftime("%Y-%m-%d %H:%M:%S")

df.to_csv('data-raw_'+formatted_time+'.csv',index=False)

In [6]:
def clean_data(df):
    df = df[df.titulo.notnull()]
    df['titulo'] = df['titulo'].str.replace(r"\n|\t","")
    df['titulo'] = df['titulo'].str.replace(r"\n|\t","")
    df['descripcion'] = df['descripcion'].str.replace(r"\n|\t","")
    df['ambiente'] = df['ambiente'].str.replace(r"\n","").str.replace(r"\t\t","; ").str.replace(r"\t","")
    df['comodidades'] = df['comodidades'].str.replace(r"\n","").str.replace(r"\t\t","; ").str.replace(r"\t","")
    df['otras caracteristicas'] = df['otras caracteristicas'].str.replace(r"\n","").str.replace(r"\t\t","; ").str.replace(r"\t","")
    df['precio'] = df['precio'].str.replace(".","")
    df['gastos comunes'] = df['gastos comunes'].str.replace("CLP","")
    df['habitaciones'] = df['habitaciones'].str.replace(r"-\s+\d\s\w*","").str.replace('Dormitorio',"").str.strip()
    df['banos'] = df['banos'].str.replace(r"-\s+\d\s\w*","").str.replace(r'Baño|s',"").str.strip()
    df['direccion'] = df['direccion'].str.replace("/",",")
    df['m2_utiles'] = df['m2_utiles'].str.replace(r"-.*$|m.","")

    return df

In [7]:
df = clean_data(df)

In [11]:
df.to_csv('clean-data'+formatted_time+'.csv',index=False)

In [5]:
#Agregar coordenadas geograficas de direcciones a Data Frame con libreria geopy
targets_address= df['direccion'].tolist()

latitude = []
longitude = []

for target in targets_address:

    location = geolocator.geocode(target)
    if location == None:
        latitude.append(np.nan)
        longitude.append(np.nan)
    else:
        latitude.append(location.latitude)
        longitude.append(location.longitude)

In [6]:
#Almacenar resultados en Data Frame
df['latitude'] = latitude
df['longitude'] = longitude

In [7]:
#Crear nuevo data frame con  direcciones validas y coordeandas geograficas no nulas 
geo_df = df[~df['latitude'].isnull()]
geo_df['City'] = 'Santiago'

In [8]:
geo_df.to_csv('santiago'+formatted_time+'.csv',index=False)